In [1]:
import pandas as pd

In [2]:
customer = pd.read_csv('customer_join.csv')

In [3]:
customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47


In [4]:
customer.shape

(4192, 18)

In [5]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        4192 non-null   object 
 1   name               4192 non-null   object 
 2   class              4192 non-null   object 
 3   gender             4192 non-null   object 
 4   start_date         4192 non-null   object 
 5   end_date           1350 non-null   object 
 6   campaign_id        4192 non-null   object 
 7   is_deleted         4192 non-null   int64  
 8   class_name         4192 non-null   object 
 9   price              4192 non-null   int64  
 10  campaign_name      4192 non-null   object 
 11  mean               4192 non-null   float64
 12  median             4192 non-null   float64
 13  max                4192 non-null   int64  
 14  min                4192 non-null   int64  
 15  routine_flg        4192 non-null   int64  
 16  calc_date          4192 

In [6]:
uselog_months = pd.read_csv('use_log_months.csv')

In [7]:
uselog_months.head(2)

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2


In [8]:
uselog_months.shape

(36842, 3)

In [9]:
uselog_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36842 entries, 0 to 36841
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   연월           36842 non-null  int64 
 1   customer_id  36842 non-null  object
 2   count        36842 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 863.6+ KB


In [10]:
year_months = list(uselog_months['연월'].unique())

In [11]:
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [12]:
uselog=pd.DataFrame()

In [13]:
for i in range(1,len(year_months)):
    tmp = uselog_months.loc[uselog_months['연월']==year_months[i]]
    tmp.rename(columns={'count':'count_0'},inplace=True)
    tmp_before = uselog_months.loc[uselog_months['연월']==year_months[i-1]]
    tmp_before.rename(columns={'count':'count_1'},inplace=True)
    del tmp_before['연월']
    tmp = pd.merge(tmp,tmp_before,on='customer_id',how='left')
    uselog = pd.concat([uselog,tmp],ignore_index=True)   

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
tmp.head(2)

,연월,customer_id,count_0,count_1
0,201903,AS002855,4,5.0
1,201903,AS008805,1,6.0


In [15]:
tmp.shape

(2953, 4)

In [16]:
uselog.head(2)

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0


In [17]:
uselog.shape

(33851, 4)

In [18]:
from dateutil.relativedelta import relativedelta

In [19]:
exit_customer = customer.loc[customer['is_deleted']==1]

In [20]:
exit_customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23


In [21]:
exit_customer.shape

(1350, 18)

In [22]:
exit_customer['exit_date']=None

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
exit_customer['exit_date'] = pd.to_datetime(exit_customer['end_date'])

In [25]:
exit_customer['exit_date']

708    2018-04-30
729    2018-04-30
730    2018-04-30
770    2018-05-31
785    2018-05-31
          ...    
4049   2019-02-28
4050   2019-03-31
4068   2019-03-31
4086   2019-03-31
4099   2019-03-31
Name: exit_date, Length: 1350, dtype: datetime64[ns]

In [26]:
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['exit_date'].iloc[i] - relativedelta(months=1)   

In [27]:
exit_customer['exit_date']

708    2018-03-30
729    2018-03-30
730    2018-03-30
770    2018-04-30
785    2018-04-30
          ...    
4049   2019-01-28
4050   2019-02-28
4068   2019-02-28
4086   2019-02-28
4099   2019-02-28
Name: exit_date, Length: 1350, dtype: datetime64[ns]

In [28]:
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m')

In [29]:
exit_customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,연월
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803


In [30]:
uselog.head(2)

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0


In [31]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33851 entries, 0 to 33850
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연월           33851 non-null  int64  
 1   customer_id  33851 non-null  object 
 2   count_0      33851 non-null  int64  
 3   count_1      32650 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.0+ MB


In [32]:
uselog['연월'] = uselog['연월'].astype(str)

In [33]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33851 entries, 0 to 33850
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연월           33851 non-null  object 
 1   customer_id  33851 non-null  object 
 2   count_0      33851 non-null  int64  
 3   count_1      32650 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.0+ MB


In [34]:
exit_uselog = pd.merge(uselog,exit_customer,on=['customer_id','연월'],how='left')

In [35]:
exit_uselog.shape

(33851, 22)

In [36]:
exit_uselog.sample(4)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
28133,201902,AS742617,2,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7622,201807,HI780815,8,NaN,XXXXX,C03,M,2018-07-08,2018-08-31,CA1,...,6000.0,일반,5.5,5.5,8.0,3.0,0.0,2018-08-31,1.0,2018-07-31
28449,201902,GD651678,9,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
30421,201902,PL856333,6,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [37]:
exit_uselog = exit_uselog.dropna(subset=['name'])

In [39]:
exit_uselog.sample(4)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
18256,201810,TS075183,3,2.0,XXXXX,C01,M,2018-04-12,2018-11-30,CA3,...,10500.0,입회비무료,4.25,4.5,8.0,2.0,1.0,2018-11-30,7.0,2018-10-30
11952,201808,PL829579,4,8.0,XXX,C01,M,2018-07-13,2018-09-30,CA1,...,10500.0,일반,5.00,4.0,8.0,3.0,1.0,2018-09-30,2.0,2018-08-30
10906,201808,IK147368,2,3.0,XXXX,C03,F,2016-10-01,2018-09-30,CA1,...,6000.0,일반,2.50,3.0,3.0,1.0,0.0,2018-09-30,23.0,2018-08-30
7714,201807,IK011787,1,1.0,XXXX,C02,F,2017-09-01,2018-08-31,CA1,...,7500.0,일반,1.80,1.0,4.0,1.0,0.0,2018-08-31,11.0,2018-07-31


In [40]:
exit_uselog.isnull().sum()

연월                    0
customer_id           0
count_0               0
count_1              52
name                  0
class                 0
gender                0
start_date            0
end_date              0
campaign_id           0
is_deleted            0
class_name            0
price                 0
campaign_name         0
mean                  0
median                0
max                   0
min                   0
routine_flg           0
calc_date             0
membership_period     0
exit_date             0
dtype: int64

In [41]:
conti_customer = customer.loc[customer['is_deleted']==0]

In [42]:
conti_uselog = pd.merge(uselog,conti_customer,on=['customer_id'],how='left')

In [43]:
conti_uselog.shape

(33851, 21)

In [45]:
conti_uselog.sample(4)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
23513,201812,IK866321,9,NaN,XX,C03,M,2018-12-10,NaN,CA3,...,야간,6000.0,입회비무료,8.250000,8.0,10.0,7.0,1.0,2019-04-30,4.0
26599,201901,IK865460,8,4.0,XXXXX,C02,F,2017-07-01,NaN,CA2,...,주간,7500.0,입회비반액할인,5.916667,5.5,9.0,4.0,1.0,2019-04-30,21.0
631,201805,GD763218,3,3.0,XX,C03,M,2015-10-01,NaN,CA1,...,야간,6000.0,일반,4.166667,4.0,6.0,3.0,1.0,2019-04-30,42.0
14480,201809,OA332826,9,6.0,XXXXX,C01,M,2018-06-07,NaN,CA2,...,종일,10500.0,입회비반액할인,7.400000,7.0,11.0,6.0,1.0,2019-04-30,10.0


In [46]:
conti_uselog.isnull().sum()

연월                       0
customer_id              0
count_0                  0
count_1               1201
name                  6429
class                 6429
gender                6429
start_date            6429
end_date             33851
campaign_id           6429
is_deleted            6429
class_name            6429
price                 6429
campaign_name         6429
mean                  6429
median                6429
max                   6429
min                   6429
routine_flg           6429
calc_date             6429
membership_period     6429
dtype: int64

In [47]:
conti_uselog = conti_uselog.dropna(subset=['name'])

In [48]:
conti_uselog.isnull().sum()

연월                       0
customer_id              0
count_0                  0
count_1                688
name                     0
class                    0
gender                   0
start_date               0
end_date             27422
campaign_id              0
is_deleted               0
class_name               0
price                    0
campaign_name            0
mean                     0
median                   0
max                      0
min                      0
routine_flg              0
calc_date                0
membership_period        0
dtype: int64

In [49]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)

In [50]:
conti_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,TS357747,8,10.0,XX,C01,M,2018-05-13,NaN,CA2,...,종일,10500.0,입회비반액할인,7.909091,8.0,10.0,7.0,1.0,2019-04-30,11.0
1,201806,TS286542,5,4.0,XXXX,C01,M,2016-01-01,NaN,CA1,...,종일,10500.0,일반,4.750000,5.0,8.0,2.0,1.0,2019-04-30,39.0
2,201805,PL343970,7,6.0,XXXXX,C01,F,2015-10-01,NaN,CA1,...,종일,10500.0,일반,5.250000,5.0,7.0,4.0,1.0,2019-04-30,42.0
3,201811,OA758034,7,9.0,XXX,C01,F,2016-09-01,NaN,CA1,...,종일,10500.0,일반,4.916667,4.5,9.0,2.0,1.0,2019-04-30,31.0
4,201811,OA955085,4,8.0,XXXXX,C01,F,2015-08-01,NaN,CA1,...,종일,10500.0,일반,4.500000,4.0,8.0,2.0,1.0,2019-04-30,44.0


In [51]:
conti_uselog = conti_uselog.drop_duplicates(subset=['customer_id'])

In [52]:
conti_uselog.shape

(2842, 21)

In [55]:
predict_data = pd.concat([conti_uselog,exit_uselog], ignore_index=True)

In [56]:
predict_data

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201812,TS357747,8,10.0,XX,C01,M,2018-05-13,NaN,CA2,...,10500.0,입회비반액할인,7.909091,8.0,10.0,7.0,1.0,2019-04-30,11.0,NaT
1,201806,TS286542,5,4.0,XXXX,C01,M,2016-01-01,NaN,CA1,...,10500.0,일반,4.750000,5.0,8.0,2.0,1.0,2019-04-30,39.0,NaT
2,201805,PL343970,7,6.0,XXXXX,C01,F,2015-10-01,NaN,CA1,...,10500.0,일반,5.250000,5.0,7.0,4.0,1.0,2019-04-30,42.0,NaT
3,201811,OA758034,7,9.0,XXX,C01,F,2016-09-01,NaN,CA1,...,10500.0,일반,4.916667,4.5,9.0,2.0,1.0,2019-04-30,31.0,NaT
4,201811,OA955085,4,8.0,XXXXX,C01,F,2015-08-01,NaN,CA1,...,10500.0,일반,4.500000,4.0,8.0,2.0,1.0,2019-04-30,44.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000.0,일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000.0,입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500.0,입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500.0,일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


In [57]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 3946 non-null   object        
 1   customer_id        3946 non-null   object        
 2   count_0            3946 non-null   int64         
 3   count_1            3691 non-null   float64       
 4   name               3946 non-null   object        
 5   class              3946 non-null   object        
 6   gender             3946 non-null   object        
 7   start_date         3946 non-null   object        
 8   end_date           1104 non-null   object        
 9   campaign_id        3946 non-null   object        
 10  is_deleted         3946 non-null   float64       
 11  class_name         3946 non-null   object        
 12  price              3946 non-null   float64       
 13  campaign_name      3946 non-null   object        
 14  mean    

In [58]:
predict_data['period']=0

In [59]:
predict_data['now_date'] = pd.to_datetime(predict_data['연월'],format='%Y%m')

In [60]:
predict_data.head(2)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201812,TS357747,8,10.0,XX,C01,M,2018-05-13,NaN,CA2,...,7.909091,8.0,10.0,7.0,1.0,2019-04-30,11.0,NaT,0,2018-12-01
1,201806,TS286542,5,4.0,XXXX,C01,M,2016-01-01,NaN,CA1,...,4.750000,5.0,8.0,2.0,1.0,2019-04-30,39.0,NaT,0,2018-06-01


In [63]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 3946 non-null   object        
 1   customer_id        3946 non-null   object        
 2   count_0            3946 non-null   int64         
 3   count_1            3691 non-null   float64       
 4   name               3946 non-null   object        
 5   class              3946 non-null   object        
 6   gender             3946 non-null   object        
 7   start_date         3946 non-null   datetime64[ns]
 8   end_date           1104 non-null   object        
 9   campaign_id        3946 non-null   object        
 10  is_deleted         3946 non-null   float64       
 11  class_name         3946 non-null   object        
 12  price              3946 non-null   float64       
 13  campaign_name      3946 non-null   object        
 14  mean    

In [62]:
predict_data['start_date']=pd.to_datetime(predict_data['start_date'])

In [65]:
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i],predict_data['start_date'][i])
    predict_data['period'] = delta.years * 12+ delta.months

In [67]:
predict_data.head(2)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201812,TS357747,8,10.0,XX,C01,M,2018-05-13,NaN,CA2,...,7.909091,8.0,10.0,7.0,1.0,2019-04-30,11.0,NaT,7,2018-12-01
1,201806,TS286542,5,4.0,XXXX,C01,M,2016-01-01,NaN,CA1,...,4.750000,5.0,8.0,2.0,1.0,2019-04-30,39.0,NaT,7,2018-06-01


In [68]:
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1               255
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [69]:
predict_data = predict_data.dropna(subset=['count_1'])

In [70]:
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2639
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2639
period                  0
now_date                0
dtype: int64

In [71]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3691 entries, 0 to 3945
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 3691 non-null   object        
 1   customer_id        3691 non-null   object        
 2   count_0            3691 non-null   int64         
 3   count_1            3691 non-null   float64       
 4   name               3691 non-null   object        
 5   class              3691 non-null   object        
 6   gender             3691 non-null   object        
 7   start_date         3691 non-null   datetime64[ns]
 8   end_date           1052 non-null   object        
 9   campaign_id        3691 non-null   object        
 10  is_deleted         3691 non-null   float64       
 11  class_name         3691 non-null   object        
 12  price              3691 non-null   float64       
 13  campaign_name      3691 non-null   object        
 14  mean    

In [72]:
target_col=['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']

In [73]:
predict_data = predict_data[target_col]

In [75]:
predict_data.head(2)

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,입회비반액할인,종일,M,10.0,1.0,7,0.0
1,일반,종일,M,4.0,1.0,7,0.0


In [76]:
predict_data = pd.get_dummies(predict_data)

In [77]:
predict_data.head(2)

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,10.0,1.0,7,0.0,0,0,1,0,1,0,0,1
1,4.0,1.0,7,0.0,1,0,0,0,1,0,0,1


In [78]:
del predict_data['gender_M']

In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [83]:
exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

In [84]:
exit.shape

(1052, 11)

In [85]:
conti.shape

(1052, 11)

In [86]:
X = pd.concat([exit,conti],ignore_index=True)

In [87]:
y= X['is_deleted']

In [88]:
del X['is_deleted']

In [91]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [107]:
model = DecisionTreeClassifier(random_state=0,max_depth=5)

In [108]:
model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5, random_state=0)

In [109]:
pred = model.predict(X_test)

In [110]:
from sklearn.metrics import accuracy_score

In [111]:
accuracy_score(y_test,pred)

0.8498098859315589

In [112]:
X_test

,count_1,routine_flg,period,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F
1091,3.0,1.0,7,1,0,0,0,1,0,0
1786,9.0,1.0,7,0,0,1,1,0,0,0
1439,6.0,1.0,7,1,0,0,1,0,0,0
745,5.0,0.0,7,0,1,0,0,1,0,1
820,6.0,1.0,7,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1029,3.0,1.0,7,0,0,1,0,1,0,0
553,3.0,0.0,7,1,0,0,0,0,1,1
1570,5.0,1.0,7,0,0,1,0,1,0,0
82,3.0,0.0,7,0,0,1,0,1,0,0


In [113]:
x_1=[[6,1,4,0,0,1,0,0,1,1]]

In [115]:
model.predict_proba(x_1)

array([[0.67567568, 0.32432432]])